## Association Rules for Heart Disease Data

Data set: [Heart Disease UCI](https://archive.ics.uci.edu/ml/datasets/Heart+Disease)  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV

from mlxtend.frequent_patterns import apriori, association_rules

### Loading Data

In [2]:
data = pd.read_csv("heart.csv", index_col=0)
data.head()

,Age,Sex,Chest_Pain,Resting_Blood_Pressure,Colestrol,Fasting_Blood_Sugar,Rest_ECG,MAX_Heart_Rate,Exercised_Induced_Angina,ST_Depression,Slope,Major_Vessels,Thalessemia,Target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       303 non-null    int64  
 1   Sex                       303 non-null    int64  
 2   Chest_Pain                303 non-null    int64  
 3   Resting_Blood_Pressure    303 non-null    int64  
 4   Colestrol                 303 non-null    int64  
 5   Fasting_Blood_Sugar       303 non-null    int64  
 6   Rest_ECG                  303 non-null    int64  
 7   MAX_Heart_Rate            303 non-null    int64  
 8   Exercised_Induced_Angina  303 non-null    int64  
 9   ST_Depression             303 non-null    float64
 10  Slope                     303 non-null    int64  
 11  Major_Vessels             303 non-null    object 
 12  Thalessemia               303 non-null    object 
 13  Target                    303 non-null    int64  
dtypes: float64

In [4]:
for col in data.columns:
    print("Column: ", col)
    print(data[col].value_counts())
    print()

Column:  Age
58    19
57    17
54    16
59    14
52    13
51    12
60    12
62    11
44    11
56    11
64    10
41    10
63     9
67     9
55     8
45     8
42     8
53     8
61     8
65     8
43     8
66     7
50     7
48     7
46     7
49     5
47     5
39     4
35     4
68     4
70     4
40     3
71     3
69     3
34     2
37     2
38     2
77     1
76     1
74     1
29     1
Name: Age, dtype: int64

Column:  Sex
1    206
0     97
Name: Sex, dtype: int64

Column:  Chest_Pain
4    144
3     86
2     50
1     23
Name: Chest_Pain, dtype: int64

Column:  Resting_Blood_Pressure
120    37
130    36
140    32
110    19
150    17
138    12
128    12
125    11
160    11
112     9
132     8
118     7
124     6
135     6
108     6
145     5
134     5
152     5
122     4
170     4
100     4
115     3
142     3
105     3
136     3
126     3
180     3
102     2
94      2
144     2
146     2
178     2
148     2
129     1
164     1
101     1
174     1
104     1
172     1
106     1
165     1
114    

The features look good for the most part. We have some missing values in **Major_Vessels** and **Thalessemia**.
So let's handle that.

### Handling missing data

In [5]:
missing_data = data[(data["Major_Vessels"] == "?") | (data["Thalessemia"] == "?")]
missing_data

,Age,Sex,Chest_Pain,Resting_Blood_Pressure,Colestrol,Fasting_Blood_Sugar,Rest_ECG,MAX_Heart_Rate,Exercised_Induced_Angina,ST_Depression,Slope,Major_Vessels,Thalessemia,Target
87,53,0,3,128,216,0,2,115,0,0.0,1,0,?,0
166,52,1,3,138,223,0,0,169,0,0.0,1,?,3,0
192,43,1,4,132,247,1,2,143,1,0.1,2,?,7,1
266,52,1,4,128,204,1,0,156,1,1.0,2,0,?,2
287,58,1,2,125,220,0,0,144,0,0.4,2,?,7,0
302,38,1,3,138,175,0,0,173,0,0.0,1,?,3,0


In [6]:
data[(data["Major_Vessels"] != "?") & (data["Thalessemia"] != "?")].shape

(297, 14)

Rather than removing the samples, we will preserve the rows and impute the missing values using the KNN classifier.

In [7]:
def impute_vals(target_var, X_missing):
    nonmissing_data = data[(data["Major_Vessels"] != "?") & (data["Thalessemia"] != "?")]
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(nonmissing_data.drop(target_var, axis=1), nonmissing_data[target_var])
    
    data.loc[X_missing.index, target_var] = knn.predict(X_missing.drop(target_var, axis=1))

In [8]:
missing_data_vess = missing_data[missing_data["Major_Vessels"] == "?"]
impute_vals("Major_Vessels", missing_data_vess)

missing_data_thal = missing_data[missing_data["Thalessemia"] == "?"]
impute_vals("Thalessemia", missing_data_thal)

In [9]:
data[(data["Major_Vessels"] != "?") & (data["Thalessemia"] != "?")].shape

(303, 14)

In [10]:
data["Major_Vessels"].value_counts()

0    179
1     66
2     38
3     20
Name: Major_Vessels, dtype: int64

In [11]:
data["Thalessemia"].value_counts()

3    167
7    118
6     18
Name: Thalessemia, dtype: int64

### Feature engineering and selection

We have several features that are numeric. In order to perform the association rules analysis, we need to convert them to categories. 

In [12]:
bins = [0, 100, 120, 140, 160, 180, 1000]
category = ['<100', '100-120', '120-140', '140-160', '160-180', '>180']
data["MAX_Heart_Rate_Cat"] = pd.cut(data['MAX_Heart_Rate'], bins, labels=category).astype('O')

In [13]:
bins = [-1, 0.01, 1, 2, 3, 10]
category = ['0', '0-1', '1-2', '2-3', '>3']
data["ST_Depression_Cat"] = pd.cut(data['ST_Depression'], bins, labels=category).astype('O')

In [14]:
bins = [0, 200, 250, 300, 1000]
category = ['<200', '200-250', '250-300', '>300']
data["Cholesterol_Cat"] = pd.cut(data['Colestrol'], bins, labels=category).astype('O')

In [15]:
bins = [0, 40, 50, 60, 70, 120]
category = ['<40', '40-50', '50-60', '60-70', '>70']
data["Age_Cat"] = pd.cut(data['Age'], bins, labels=category).astype('O')

In [16]:
bins = [0, 120, 140, 160, 180, 500]
category = ['<120', '120-140', '140-160', '160-180', '>180']
data["BP_Cat"] = pd.cut(data['Resting_Blood_Pressure'], bins, labels=category).astype('O')

In [17]:
data_binned = pd.get_dummies(data.drop(["MAX_Heart_Rate", "ST_Depression", "Colestrol", "Age", "Resting_Blood_Pressure"], axis=1))

While we could use all of the features for the association rules analysis, we will select only those that are important. To determine the importance we will use the Random Forest Classifier.

In [18]:
X = data_binned.drop("Target", axis=1)
y = data["Target"]

In [19]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
rfe = RFECV(estimator=RandomForestClassifier(n_estimators=200), cv=cv)

In [20]:
rfe.fit(X, y)

RFECV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=None),
      estimator=RandomForestClassifier(n_estimators=200))

In [21]:
rfe_df = pd.DataFrame(columns=["Feature", "Selected", "Rank", "Score"], 
                      data=zip(X.columns, rfe.support_, rfe.ranking_, rfe.grid_scores_))
rfe_df.sort_values(by="Rank", ascending=True)

,Feature,Selected,Rank,Score
0,Sex,True,1,0.530237
36,BP_Cat_<120,True,1,0.568925
33,BP_Cat_120-140,True,1,0.578889
16,MAX_Heart_Rate_Cat_160-180,True,1,0.574390
15,MAX_Heart_Rate_Cat_140-160,True,1,0.573260
14,MAX_Heart_Rate_Cat_120-140,True,1,0.573260
24,Cholesterol_Cat_200-250,True,1,0.586612
12,Thalessemia_7,True,1,0.561075
25,Cholesterol_Cat_250-300,True,1,0.576667
21,ST_Depression_Cat_1-2,True,1,0.587596


In [22]:
selected_features = rfe_df[rfe_df["Selected"] == True]["Feature"]
selected_features

0                            Sex
1                     Chest_Pain
3                       Rest_ECG
4       Exercised_Induced_Angina
5                          Slope
6                Major_Vessels_0
7                Major_Vessels_1
8                Major_Vessels_2
10                 Thalessemia_3
12                 Thalessemia_7
14    MAX_Heart_Rate_Cat_120-140
15    MAX_Heart_Rate_Cat_140-160
16    MAX_Heart_Rate_Cat_160-180
20         ST_Depression_Cat_0-1
21         ST_Depression_Cat_1-2
24       Cholesterol_Cat_200-250
25       Cholesterol_Cat_250-300
29                 Age_Cat_50-60
30                 Age_Cat_60-70
33                BP_Cat_120-140
36                   BP_Cat_<120
Name: Feature, dtype: object

In [23]:
data_binned[selected_features]

,Sex,Chest_Pain,Rest_ECG,Exercised_Induced_Angina,Slope,Major_Vessels_0,Major_Vessels_1,Major_Vessels_2,Thalessemia_3,Thalessemia_7,...,MAX_Heart_Rate_Cat_140-160,MAX_Heart_Rate_Cat_160-180,ST_Depression_Cat_0-1,ST_Depression_Cat_1-2,Cholesterol_Cat_200-250,Cholesterol_Cat_250-300,Age_Cat_50-60,Age_Cat_60-70,BP_Cat_120-140,BP_Cat_<120
0,1,1,2,0,3,1,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
1,1,4,2,1,2,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,0
2,1,4,2,1,2,0,0,1,0,1,...,0,0,0,0,1,0,0,1,0,1
3,1,3,0,0,3,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
4,0,2,2,0,1,1,0,0,1,0,...,0,1,0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,1,1,0,0,2,1,0,0,0,1,...,0,0,0,1,0,1,0,0,0,1
299,1,4,0,0,2,0,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
300,1,4,0,1,2,0,1,0,0,1,...,0,0,0,1,0,0,1,0,1,0
301,0,2,2,0,2,0,1,0,1,0,...,0,1,0,0,1,0,1,0,1,0


We have a number of columns representing categories that are encoded as numeric. Let's convert them into categories and then into indicator variables.

In [24]:
numeric_cols = ['Sex', 'Chest_Pain', 'Fasting_Blood_Sugar', 'Rest_ECG', 'Exercised_Induced_Angina', 'Slope', 'Target']
data_binned[numeric_cols] = data_binned[numeric_cols].astype('O')

In [25]:
df = pd.get_dummies(data_binned)
df.head()

,Major_Vessels_0,Major_Vessels_1,Major_Vessels_2,Major_Vessels_3,Thalessemia_3,Thalessemia_6,Thalessemia_7,MAX_Heart_Rate_Cat_100-120,MAX_Heart_Rate_Cat_120-140,MAX_Heart_Rate_Cat_140-160,...,Exercised_Induced_Angina_0,Exercised_Induced_Angina_1,Slope_1,Slope_2,Slope_3,Target_0,Target_1,Target_2,Target_3,Target_4
0,1,0,0,0,0,1,0,0,0,1,...,1,0,0,0,1,1,0,0,0,0
1,0,0,0,1,1,0,0,1,0,0,...,0,1,0,1,0,0,0,1,0,0
2,0,0,1,0,0,0,1,0,1,0,...,0,1,0,1,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,1,1,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0


### Discovering interesting relations

In [26]:
frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)
frequent_itemsets.sort_values(by="support", ascending=False)

,support,itemsets
13,0.851485,(Fasting_Blood_Sugar_0)
11,0.679868,(Sex_1)
16,0.673267,(Exercised_Induced_Angina_0)
0,0.590759,(Major_Vessels_0)
53,0.577558,"(Fasting_Blood_Sugar_0, Exercised_Induced_Angi..."
...,...,...
72,0.300330,"(Thalessemia_3, Slope_1, Exercised_Induced_Ang..."
74,0.300330,"(Chest_Pain_4, Sex_1, Fasting_Blood_Sugar_0)"
80,0.300330,"(Major_Vessels_0, Thalessemia_3, Fasting_Blood..."
82,0.300330,"(Major_Vessels_0, Target_0, Thalessemia_3, Exe..."


In [27]:
ar = association_rules(frequent_itemsets, metric="lift")
interesting_rules = ar[(ar["lift"] > 1.5) & (ar["conviction"] > 2)].sort_values(by="conviction", ascending=False)
interesting_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
216,"(Major_Vessels_0, Thalessemia_3, Exercised_Ind...",(Target_0),0.330033,0.541254,0.300330,0.910000,1.681280,0.121698,5.097176
97,"(Major_Vessels_0, Thalessemia_3)",(Target_0),0.389439,0.541254,0.343234,0.881356,1.628359,0.132449,3.866572
202,"(Major_Vessels_0, Thalessemia_3, Fasting_Blood...",(Target_0),0.353135,0.541254,0.310231,0.878505,1.623091,0.119095,3.775831
121,"(Major_Vessels_0, Exercised_Induced_Angina_0)",(Target_0),0.442244,0.541254,0.376238,0.850746,1.571806,0.136871,3.073597
245,"(Exercised_Induced_Angina_0, Thalessemia_3, Fa...",(Target_0),0.389439,0.541254,0.326733,0.838983,1.550072,0.115947,2.849053
146,"(Thalessemia_3, Exercised_Induced_Angina_0)",(Target_0),0.445545,0.541254,0.372937,0.837037,1.546477,0.131784,2.815032
230,"(Major_Vessels_0, Exercised_Induced_Angina_0, ...",(Target_0),0.396040,0.541254,0.330033,0.833333,1.539634,0.115675,2.752475
205,"(Major_Vessels_0, Thalessemia_3)","(Target_0, Fasting_Blood_Sugar_0)",0.389439,0.465347,0.310231,0.796610,1.711864,0.129007,2.628713
219,"(Major_Vessels_0, Thalessemia_3)","(Target_0, Exercised_Induced_Angina_0)",0.389439,0.465347,0.300330,0.771186,1.657230,0.119106,2.336634
234,"(Major_Vessels_0, Exercised_Induced_Angina_0)","(Target_0, Fasting_Blood_Sugar_0)",0.442244,0.465347,0.330033,0.746269,1.603684,0.124236,2.107164


In [28]:
interesting_rules["antecedents"].values

array([frozenset({'Major_Vessels_0', 'Thalessemia_3', 'Exercised_Induced_Angina_0'}),
       frozenset({'Major_Vessels_0', 'Thalessemia_3'}),
       frozenset({'Major_Vessels_0', 'Thalessemia_3', 'Fasting_Blood_Sugar_0'}),
       frozenset({'Major_Vessels_0', 'Exercised_Induced_Angina_0'}),
       frozenset({'Exercised_Induced_Angina_0', 'Thalessemia_3', 'Fasting_Blood_Sugar_0'}),
       frozenset({'Thalessemia_3', 'Exercised_Induced_Angina_0'}),
       frozenset({'Major_Vessels_0', 'Exercised_Induced_Angina_0', 'Fasting_Blood_Sugar_0'}),
       frozenset({'Major_Vessels_0', 'Thalessemia_3'}),
       frozenset({'Major_Vessels_0', 'Thalessemia_3'}),
       frozenset({'Major_Vessels_0', 'Exercised_Induced_Angina_0'}),
       frozenset({'Thalessemia_3', 'Exercised_Induced_Angina_0'})],
      dtype=object)

Thalessemia_3, Major_Vessels_0, Exercised_Induced_Angina_0, Fasting_Blood_Sugar_0 and Target_0 are all strongly related to each other. Presence of some of these characteristics says a lot about the other.

Looking at the association rules, the main conclusion is: 
If patients have the following conditions: 
- no Thalessemia defects (Thalessemia_3 = normal)
- no Angina induced by exercising (Exercised_Induced_Angina_0) 
- fasting blood sugar is less than 120 mg/dl (Fasting_Blood_Sugar_0)
- the number of vessels colored by fluoroscopy is 0 (Major_Vessels_0) 

then it's more likely that they don't have a heart disease (Target_0).